In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Configura geolocalizador
geolocator = Nominatim(user_agent="meu_app", timeout=3)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

df = pd.read_excel("base.xlsx")

df

,Município,1986,1986.1,1986.2,1987,1987.1,1987.2,1988,1988.1,1988.2,...,2020.2,2021,2021.1,2021.2,2022,2022.1,2022.2,2023,2023.1,2023.2
0,NaN,Cera,Pó,Fibra,Cera,Pó,Fibra,Cera,Pó,Fibra,...,Fibra,Cera,Pó,Fibra,Cera,Pó,Fibra,Cera,Pó,Fibra
1,Alta Floresta D'Oeste (RO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ariquemes (RO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cabixi (RO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cacoal (RO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358,Vianópolis (GO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5359,Vicentinópolis (GO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5360,Vila Boa (GO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5361,Vila Propício (GO),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
anos = []
ano = 2019

for i in range(1,6):
    anos.append(str(ano))
    ano = 2019 + i

print(anos)

['2019', '2020', '2021', '2022', '2023']


In [7]:
for i in anos:
    df_novo = df[["Município",i,i+".1",i+".2"]]
    for n in range(3):
        adicional_ano = ["",".1",".2"]
        produto = ["Cera","Pó","Fibra"]
        df_novo_especifico = df[["Município", i + adicional_ano[n]]].copy()
        df_novo_especifico = df_novo_especifico.drop(index=0)
        df_novo_especifico = df_novo_especifico.rename(columns={i + adicional_ano[n]: "Produção(toneladas)"})
        df_novo_especifico = df_novo_especifico[df_novo_especifico["Produção(toneladas)"] != 0]

        # Função segura para pegar coordenadas
        def get_coords_safe(municipio):
            try:
                location = geocode(f"{municipio}, Brasil")
                if location:
                    return pd.Series([location.latitude, location.longitude])
                else:
                    return pd.Series([None, None])
            except Exception as e:
                print(f"Erro em {municipio}: {e}")
                return pd.Series([None, None])

        # Aplica ao DataFrame
        df_novo_especifico[['Latitude', 'Longitude']] = df_novo_especifico['Município'].apply(get_coords_safe)

        df_novo_especifico.loc[:, "Produto"] = produto[n]
        df_novo_especifico["Ano"] = i
        df_novo_especifico['Estado_UF'] = df_novo_especifico['Município'].str.extract(r'\((\w{2})\)')
        df_novo_especifico.to_excel(produto[n]+"_"+i+".xlsx", index=False)
print("Deu certo!!!")

Deu certo!!!
